In [16]:
import pylab
import h5py
import math
import array

import numpy
from numpy import *
import numpy as np
from pycbc.types import TimeSeries, FrequencySeries
from pycbc.waveform import get_td_waveform, get_fd_waveform
from pycbc.waveform.waveform_modes import get_td_waveform_modes
from pycbc import types, fft, waveform
import lal
from scipy import interpolate
from scipy.interpolate import interp1d
from lal import MSUN_SI, MTSUN_SI, G_SI, PC_SI, C_SI, PI
from pycbc.filter import match
from pycbc.psd import aLIGOZeroDetHighPower
from tqdm import tqdm#

#from matplotlib import rcParams
#rcParams.update({'figure.autolayout': True})
import matplotlib as mpl
#mpl.rcParams['figure.dpi'] = 200
#plt.rcParams["font.family"] = "monospace"
 
from matplotlib import gridspec
from matplotlib import ticker

import matplotlib.pyplot as plt
#plt.style.reload_library()
#plt.style.use(['science','notebook'])



#pylab.rc('xtick', labelsize=18) 
#pylab.rc('ytick', labelsize=18) 
#pylab.rc('axes', labelsize=16)
#pylab.rc('legend', fontsize=15)


In [17]:
# Eq. (4.17a, 4.17b), Pg. 18, Moore et al (2016)

def epsilon(xi, eta):
    return(( 1 + ( ( -2833/2016 + 197/72 * eta ) * ( xi )**( 2/3 ) + 
                  ( -377/144 * np.pi * xi + ( ( 77006005/24385536 + ( -1143767/145152 * eta + 
 	 43807/10368 * ( eta )**( 2 ) ) ) * ( xi )**( 4/3 ) + ( np.pi * ( 9901567/1451520 + 
 	 -202589/362880 * eta ) * ( xi )**( 5/3 ) + ( xi )**( 2 ) * ( -33320661414619/386266890240 + 
 	 ( 3317/252 * EulerGamma + ( 180721/41472 * ( np.pi )**( 2 ) + ( ( 161339510737/8778792960 + 
 	 3977/2304 * ( np.pi )**( 2 ) ) * eta + ( -359037739/20901888 * ( eta )**( 2 ) + 
      ( 10647791/2239488 * ( eta )**( 3 ) + ( -87419/3780 * np.log( 2 ) + 
 	 ( 26001/1120 * np.log( 3 ) + 3317/504 * np.log( 16 * ( xi )**( 2/3 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ))

In [18]:
# "Hinder+ modified all 20 simulations SEOBNRv4 model, full frequency range --Feb 16
g=open('/home/pratul/Downloads/Project/Analytical fits/New results/New_tshift_H+modified_20hyb_Feb16.txt',"r") 
lines=g.readlines() 
A=[] 
for x in lines: 
    A.append(float(x.split()[1])) 
g.close()

def tshift_Hinsp(q,e,l): 
    return A[0] + A[1]*q + A[2]*q**2 + A[3]*e + A[4]*e**2 + A[5]*e*q + A[6]*(e**2)*q + A[7]*(e)*math.cos(l + A[8]) + A[9]*(e**1)*(q**1)*math.cos(e*l + A[10]) + A[11]*(e**2)*math.cos(l + A[12])

g=open('/home/pratul/Downloads/Project/Analytical fits/New results/New_tamp_H+modified_20hyb_Feb16.txt',"r") 
lines=g.readlines() 
B=[] 
for x in lines: 
    B.append(float(x.split()[1])) 
g.close()

def tamp_Hinsp(eta,e,l): 
    return B[0] + B[1]*eta + B[2]*eta**2 + B[3]*e + B[4]*e**2 + B[5]*e*eta + B[6]*(e**2)*eta + B[7]*e*math.cos(l + B[8]) + B[9]*(e**1)*(eta**1)*math.cos(l + B[10]) + B[11]*(e**2)*math.cos(e*l + B[12])

g=open('/home/pratul/Downloads/Project/Analytical fits/New results/New_tfreq_H+modified_20hyb_Feb16.txt',"r") 
lines=g.readlines() 
C=[] 
for x in lines: 
    C.append(float(x.split()[1])) 
g.close()

def tfreq_Hinsp(eta,e,l): 
    return C[0] + C[1]*eta + C[2]*eta**2 + C[3]*e + C[4]*e**2 + C[5]*e*eta + C[6]*(e**2)*eta + C[7]*e*eta*math.cos(l + C[8]) + C[9]*(e)*math.cos(l + C[10]) + C[11]*(e**2)*math.cos(e*l + C[12])

In [19]:
def sph_harmonics(inc,ell):
    L=ell
    #inc = 10
    theta = inc
    for l in range(L,L+1):

        for m in range(-l,l+1):
            dlm = 0;
            k1 = max([0, m-2]);
            k2 = min([l+m, l-2]);

            #if(m==l or m==l-1):
            for k in range(k1,k2+1):
                A = []; B = []; cosTerm = []; sinTerm = []; dlmTmp = [];

                A = (-1)**k*math.sqrt(math.factorial(l+m)*math.factorial(l-m)*math.factorial(l+2)*math.factorial(l-2));
                B = math.factorial(k)*math.factorial(k-m+2)*math.factorial(l+m-k)*math.factorial(l-k-2);

                cosTerm = pow(math.cos(theta/2), 2*l+m-2*k-2);
                sinTerm = pow(math.sin(theta/2), 2*k-m+2);

                dlmTmp = (A/B)*cosTerm*sinTerm;
                dlm = dlm+dlmTmp

            Ylm = math.sqrt((2*l+1)/(4*math.pi))*dlm
            #print('l:',l,'m:',m,'\t Y_lm:',Ylm)
            if m==ell:
                #globals()['sph' + str(l) + str(m)] = Ylm
                #print('l:',l,'m:',m,'\t Y_lm:',Ylm)
                sphlm = Ylm
            elif m==-ell:
                #globals()['sph' + str(l) + '_' + str(abs(m))] = Ylm
                #print('l:',l,'m:',m,'\t Y_lm:',Ylm)
                sphl_m = Ylm
            else:
                continue
    return sphlm, sphl_m

In [20]:
sph1,sph2 = sph_harmonics(0,2)
#print(sph1,sph2)

0.6307831305050401 0.0


In [21]:
def xi(x):
    return x**(3/2)

def xconv(f,M):
    return (PI*M*MTSUN_SI*f)**(2/3)  #22 mode conversion

def fconv(x,M):
    return x**(3/2)/(PI*M*MTSUN_SI)  #22 mode conversion


In [22]:
xconv(16.68,30)

0.03913912683431347

In [23]:
def eccmodel(Mass,q0,e0,l0,fmin,inclination=0,d=1,delta_t=1./4096,modes=[[2,2]]):
    
    #delta_t=0.00015208911520102518
    #delta_t = 1/2**20
    ell = []
    numrows = len(modes)
    numcols = len(modes[0])
    for i in range(0,numrows):
        l = modes[i][0]
        m = modes[i][1]
        ell.append(l)
    angle = inclination
    waveform = {}
    count = 0
    el = 2
    if el in ell:
        mode_data = {}
        mode_data['hp'], mode_data['hc'], mode_data['t'] = MODEL22(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        waveform['l2_m2'] = mode_data
        count = count + 1

    el = 3
    if el in ell:
        mode_data = {}
        mode_data['hp'], mode_data['hc'], mode_data['t'] = MODEL33(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        waveform['l3_m3'] = mode_data
        count = count + 1
        
    el = 4
    if el in ell:
        mode_data = {}
        mode_data['hp'], mode_data['hc'], mode_data['t'] = MODEL44(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        waveform['l4_m4'] = mode_data
        count = count + 1
        
    el = 5
    if el in ell:
        mode_data = {}
        mode_data['hp'], mode_data['hc'], mode_data['t'] = MODEL55(Mass,q0,e0,l0,fmin,angle,d,delta_t)
        waveform['l5_m5'] = mode_data
        count = count + 1
        
    len_max_mode = '0'
    len_max = 0
    for mode in waveform.keys():
        if len(waveform[mode]['t'])>len_max:
            len_max_mode = mode
            len_max = len(waveform[mode]['t'])
            
    for mode in waveform.keys():
        if mode != len_max_mode:
            waveform[mode]['hp'].resize(len_max)
            waveform[mode]['hc'].resize(len_max)
            
    hp=0
    hc=0
    time=waveform[len_max_mode]['t']
    for mode in waveform.keys():
        hp = hp + waveform[mode]['hp']
        hc = hc + waveform[mode]['hc']
        
    hplus = TimeSeries(hp,delta_t,epoch=time[0])
    hcross = TimeSeries(hc,delta_t,epoch=time[0])
    
    return hplus, hcross

In [24]:
def TT2_INSP_Eber22(M0,q,e0,l0,flow,inc,d0,delta_t):
    
    eta=neu=nu=q/(1+q)**2
    G=c=M=d=1
    M2=M/(1+q)
    M1=M2*q
    Delta=math.sqrt(1-(4*neu))
    eta=nu=neu
    gamma=EulerGamma=0.577215664901
    mode2polfac=(5/(64*np.pi))**(1/2)
    
    conv=M*MTSUN_SI
    M_SI=M * MSUN_SI
    D_SI=(10**(6)) * PC_SI * d
    
    xlow = ((M0*MTSUN_SI*math.pi*flow)**(2/3))
    f_low = (xlow**(3/2)/(M*MTSUN_SI*math.pi))
    
    %run GW_functions.ipynb
    
    x=xlow
    v=math.sqrt(x)
    
    xie=v**3
    
    if delta_t>=1/2**14:
        del_t = 1/2**14
    elif delta_t<1/2**14 and delta_t>=1/2**16:
        del_t = 1/2**16
    elif delta_t<1/2**16 and delta_t>=1/2**18:
        del_t = 1/2**18
    else:
        del_t = 1/2**20
        
    #phase_EccTD, tVec_PN = PNparams(M,q,d,f_low,e0,del_t)
    
    tVec_PN = np.arange(-35000, -500, 10)   #Turn on for omkar's code
    tC_NR = 0
    
    x0=xlow
    xi0=x0**(3/2)
    v0=xi0**(1/3)
   
    theta=((5*M/(eta))**(1/8))*(tC_NR-tVec_PN)**(-1/8)
    theta0=((5*M/(eta))**(1/8))*(tC_NR-tVec_PN[0])**(-1/8)
    fVec=x_from_t(theta, theta0, e0, M, eta)
    
    plotIdx2=np.nonzero(fVec>=0)
    fVec=fVec[plotIdx2]
    xiVec=(np.pi*M*fVec)
    xVec=xiVec**(2/3)
    vVec=xiVec**(1/3)
    xband=np.where(xVec<=1/6)
    xVec = xVec[xband]
    maxPNidx = len(xVec)
    tVec_PN=tVec_PN[:maxPNidx]
    
    lp=2
    mp=2
    
    j=0
    h22=[]
    h2_2=[]
    
    #W_term = []; XIE = []
    for i in xVec:    #tqdm(xVec) for status bar
        v=math.sqrt(i)
        v0=math.sqrt(x0)
        xie=v**3
        xi0=v0**3
        l=mean_anomaly(xie, xi0, l0, eta, e0)
        e=e0*(xi0/xie)**(19/18)*epsilon(xie, eta)/epsilon(xi0, eta)
        
        j=j+1
        xi=l   #use xi for amplitude (xie is being used for v**3)
        x=i
        #psi = 2*phase_TT2(x**0.5, x0**0.5, e0, 0, eta) + 2*W(xie, xi0, eta, e0, l) #Old TT2 phase (Abhishek)
        psi= mp*New_phase_TT2(x, x0, e0, 0.0, 0.0, 0.0, M, nu, Delta)+mp*W(xie, xi0, eta, e0, l) # New TT2 phase (Omkar)
        #psi = phase_EccTD[j]   # LAL approximant
        #W_term.append(W(xie,xi0,eta,e0,l)); XIE.append(xie/(PI*2.8*MTSUN_SI))
        h=amplitude_22(xi,x,nu,Delta,e) #### 22 mode requires additional  eccentricity input
        
        hlm=8*math.sqrt(math.pi/5)*M*neu*i*h*((np.e)**(complex(0,-1)*mp*psi/2))/d
        hl_m=8*math.sqrt(math.pi/5)*M*neu*i*h*((np.e)**(complex(0,+1)*mp*psi/2))/d
        
        h22.append(hlm)
        h2_2.append(hl_m)
        
    
    
    conv_t = M0*MTSUN_SI
    conv_h = G_SI*M0*MSUN_SI/(10**6 * PC_SI * d0)/C_SI/C_SI
    
    sph22, sph2_2 = sph_harmonics(inc,lp)
    
    h = np.multiply(h22,sph22) #+np.multiply(h2_2,sph2_2)
    hp=(np.real(h)) 
    hc=(np.imag(h))
    
    #print(tVec_PN)
    time = tVec_PN - tVec_PN[-1]

    mode2polfac=(5/(64*np.pi))**(1/2)  
    
    hp = np.array(hp) * conv_h
    hc = np.array(hc) * conv_h
    time = tVec_PN * conv_t
    
    hp_intrp = interp1d(time, hp, kind='cubic',fill_value='extrapolate')
    hc_intrp = interp1d(time, hc, kind='cubic',fill_value='extrapolate')
    t_intrp = np.arange(time.min(), time.max(), delta_t)
    hp_intrp = hp_intrp(t_intrp)
    hc_intrp = hc_intrp(t_intrp)
    
    #plt.plot(XIE,W_term)
    return np.array(hp_intrp), np.array(hc_intrp), np.array(t_intrp)

In [25]:
def TT2_MODEL22(m,q0,e0,l0,fmin,angle,d,delta_t):
    
    M=m
    M1=q0*M/(1+q0)
    M2=M/(1+q0)
    eta=q0/(1+q0)**2
    M_SI=M*MSUN_SI
    D_SI=(10**(6))*PC_SI
    inc = angle
    angle = (np.pi/180)*angle
        
    mode2polfac=4*(5/(64*np.pi))**(1/2)
         
    
    hp, hc, tinsp = TT2_INSP_Eber22(M,q0,e0,l0,fmin,angle,d,delta_t) 
    
        
    #Circular IMR
    sp, sc = get_td_waveform(approximant='SEOBNRv4', mass1=M1, mass2=M2, delta_t=delta_t, f_lower=fmin)

    h22IMR = sp+1j*sc
    
    tshift = -tshift_Hinsp(q0,e0,l0)*M*MTSUN_SI
    
    #print(-tshift_Hinsp(q0,e0,l0))
    tmin = max(tinsp[0]-tshift, sp.sample_times[0])
    
    
    tImr_intrp = np.arange(tmin, sp.sample_times[-1], delta_t)
    tImr = tImr_intrp
    H_intrp = interp1d(sp.sample_times, h22IMR, kind='cubic', fill_value='extrapolate')
    h22Imr = H_intrp(tImr_intrp)
    
    
    
    
    
    #Interpolation Ebersold
    hp_intrp = interp1d(tinsp-tshift, hp, kind='cubic',fill_value='extrapolate')
    hc_intrp = interp1d(tinsp-tshift, hc, kind='cubic',fill_value='extrapolate')
    tEcc_intrp = np.arange(tmin, tinsp[-1]-tshift, delta_t)
    hp_intrp = hp_intrp(tEcc_intrp)
    hc_intrp = hc_intrp(tEcc_intrp)
    tEcc = tEcc_intrp
    hpEcc = hp_intrp
    hcEcc = hc_intrp
    h22Ecc = hpEcc + 1j*hcEcc
    
    phaseEcc = np.unwrap(np.angle(h22Ecc)*2)/2
    phaseImr = np.unwrap(np.angle(h22Imr)*2)/2
    dphase = phaseEcc[0] - phaseImr[0]
    hp_new = real(h22Ecc * exp(-1j * dphase))
    hc_new = imag(h22Ecc * exp(-1j * dphase))
    
    phase_new = np.unwrap(np.angle(hp_new-1j*hc_new)*2)/2
    
    phaseEcc = phase_new 
    phaseImr = phaseImr 
    h22Ecc_new = (hp_new+1j*hc_new)
    
    arg = np.argmin(abs(tEcc-tamp_Hinsp(eta,e0,l0)*M*MTSUN_SI))
    Idxjoin = arg
    
    t_amp = tEcc[Idxjoin] - 500*M*MTSUN_SI
    idxstr = np.argmin(abs(tEcc-t_amp))
    
    #Amplitude Model
    amp=[]
    count=0
    length=Idxjoin-idxstr
    
    for i in range(idxstr,Idxjoin):
        amp.append(((length-count)*abs(h22Ecc_new[i])+count*abs(h22Imr[i]))/length)
        count=count+1
    
    t_model=np.concatenate((tEcc[0:Idxjoin],tImr[Idxjoin:len(tImr)]))
    h22amp=np.concatenate((abs(h22Ecc_new[0:idxstr]),amp))
    h22amp_model=np.concatenate((h22amp,abs(h22Imr[Idxjoin:len(h22Imr)])))    
    
    omegaEcc = (M*MTSUN_SI/delta_t)*(np.gradient(phaseEcc))
    omegaImr = (M*MTSUN_SI/delta_t)*(np.gradient(phaseImr))
    
    tjoin0 = tfreq_Hinsp(eta,e0,l0)
    #print(tjoin0)
    tjoin = tjoin0 * M * MTSUN_SI
    fjoin = np.argmin(abs(tEcc-tjoin))
        
    #frequency model
    tstop = min(tEcc[-1],-30*M*MTSUN_SI)
    lst=np.argmin(abs(tEcc-tstop))
    
    indx=lst - fjoin
    a0 = []
    n = indx-1
    k = 0
    for i in range(fjoin,fjoin+indx):
        a0.append(((n-k)*omegaEcc[i]+k*omegaImr[i])/n)
        k=k+1
    
    f1 = np.concatenate((omegaEcc[0:fjoin],a0))
    frequency_model = np.concatenate((f1,omegaImr[fjoin+indx:len(omegaImr)]))
    phase_f_model = np.cumsum(frequency_model)/(M*MTSUN_SI/delta_t)
    phase_f_model = phase_f_model - phase_f_model[0] + phaseEcc[0]
    
    hp_f_model = h22amp_model * np.cos(phase_f_model)
    hc_f_model = h22amp_model * np.sin(phase_f_model)
    
    ht = (mode2polfac/4)*(((1+math.cos(angle))**2 * (hp_f_model - 1j*hc_f_model)) + ((1-math.cos(angle))**2 * (hp_f_model + 1j*hc_f_model)))
    hplus = np.real(ht)
    hcross = np.imag(ht)
    
    hp_model_TS = TimeSeries(hplus,delta_t)
    hc_model_TS = TimeSeries(hcross,delta_t)
    #print(hplus)   
    return hp_model_TS, hc_model_TS, t_model#, frequency_model
    

In [26]:
#MODEL22(30,3,0.3,3.005,13.75,0,1,1./4096)
#MODEL22(30,2,0.172,-0.181,16.68,0,1,1./4096)
#MODEL22(30,1,0.173,1.423,16.53,0,1,1./4096)


In [1]:
def PNparams(M,q,d,f_low,e0,delta_t):
    M2=M/(1+q)
    M1=M2*q
    hpVec_PN, hcVec_PN = get_td_waveform(approximant='EccentricTD', mass1=M1, mass2=M2, 
                            delta_t=delta_t, 
                            f_lower=f_low, 
                            eccentricity=e0,
                            distance=d)
    
    modetopolfac=4*(5/(64*np.pi))**(1/2)  #conversion factor between mode amplitude and strain amplitude
                                          # check 0704.3764 equation 7 for details
    M_SI = M * MSUN_SI
    D_SI = 10**6 * PC_SI * d
    phase_EccTD = waveform.utils.phase_from_polarizations(hpVec_PN/(modetopolfac*(G_SI*M_SI/D_SI/C_SI/C_SI)), hcVec_PN/(modetopolfac*(G_SI*M_SI/D_SI/C_SI/C_SI)))
    tVec_PN=hpVec_PN.sample_times/(M*MTSUN_SI)
    
    return phase_EccTD, tVec_PN

In [ ]:
def Ecc_INSP_Eber22(M0,q,e0,l0,flow,inc,d0,delta_t):
    
    eta=neu=nu=q/(1+q)**2
    G=c=M=d=1
    M2=M/(1+q)
    M1=M2*q
    Delta=math.sqrt(1-(4*neu))
    eta=nu=neu
    gamma=EulerGamma=0.577215664901
    mode2polfac=(5/(64*np.pi))**(1/2)
    
    conv=M*MTSUN_SI
    M_SI=M * MSUN_SI
    D_SI=(10**(6)) * PC_SI * d
    
    xlow = ((M0*MTSUN_SI*math.pi*flow)**(2/3))
    f_low = (xlow**(3/2)/(M*MTSUN_SI*math.pi))
    
    %run GW_functions.ipynb
    
    x=xlow
    v=math.sqrt(x)
    
    xie=v**3
    
    if delta_t>=1/2**14:
        del_t = 1/2**14
    elif delta_t<1/2**14 and delta_t>=1/2**16:
        del_t = 1/2**16
    elif delta_t<1/2**16 and delta_t>=1/2**18:
        del_t = 1/2**18
    else:
        del_t = 1/2**20
        
    phase_EccTD, tVec_PN = PNparams(M,q,d,f_low,e0,del_t)
    
    tC_NR = 0
    
    x0=xlow
    xi0=x0**(3/2)
    v0=xi0**(1/3)
   
    theta=((5*M/(eta))**(1/8))*(tC_NR-tVec_PN)**(-1/8)
    theta0=((5*M/(eta))**(1/8))*(tC_NR-tVec_PN[0])**(-1/8)
    fVec=x_from_t(theta, theta0, e0, M, eta)
    
    plotIdx2=np.nonzero(fVec>=0)
    fVec=fVec[plotIdx2]
    xiVec=(np.pi*M*fVec)
    xVec=xiVec**(2/3)
    vVec=xiVec**(1/3)
    xband=np.where(xVec<=1/6)
    xVec = xVec[xband]
    maxPNidx = len(xVec)
    tVec_PN=tVec_PN[:maxPNidx]
    
    lp=2
    mp=2
    
    j=0
    h22=[]
    h2_2=[]
    
    for i in xVec:    #tqdm(xVec) for status bar
        v=math.sqrt(i)
        v0=math.sqrt(x0)
        xie=v**3
        xi0=v0**3
        l=mean_anomaly(xie, xi0, l0, eta, e0)
        e=e0*(xi0/xie)**(19/18)*epsilon(xie, eta)/epsilon(xi0, eta)
        
        j=j+1
        xi=l   #use xi for amplitude (xie is being used for v**3)
        x=i
        psi = phase_EccTD[j]   # LAL approximant
        h=amplitude_22(xi,x,nu,Delta,e) #### 22 mode requires additional  eccentricity input
        
        hlm=8*math.sqrt(math.pi/5)*M*neu*i*h*((np.e)**(complex(0,-1)*mp*psi/2))/d
        hl_m=8*math.sqrt(math.pi/5)*M*neu*i*h*((np.e)**(complex(0,+1)*mp*psi/2))/d
        
        h22.append(hlm)
        h2_2.append(hl_m)
        
    
    
    conv_t = M0*MTSUN_SI
    conv_h = G_SI*M0*MSUN_SI/(10**6 * PC_SI * d0)/C_SI/C_SI
    
    sph22, sph2_2 = sph_harmonics(inc,lp)
    
    h = np.multiply(h22,sph22) #+np.multiply(h2_2,sph2_2)
    hp=(np.real(h)) 
    hc=(np.imag(h))
    
    #print(tVec_PN)
    time = tVec_PN - tVec_PN[-1]

    mode2polfac=(5/(64*np.pi))**(1/2)  
    
    hp = np.array(hp) * conv_h
    hc = np.array(hc) * conv_h
    time = tVec_PN * conv_t
    
    hp_intrp = interp1d(time, hp, kind='cubic',fill_value='extrapolate')
    hc_intrp = interp1d(time, hc, kind='cubic',fill_value='extrapolate')
    t_intrp = np.arange(time.min(), time.max(), delta_t)
    hp_intrp = hp_intrp(t_intrp)
    hc_intrp = hc_intrp(t_intrp)
    
    return np.array(hp_intrp), np.array(hc_intrp), np.array(t_intrp)

In [ ]:
def Ecc_MODEL22(m,q0,e0,l0,fmin,angle,d,delta_t):
    
    M=m
    M1=q0*M/(1+q0)
    M2=M/(1+q0)
    eta=q0/(1+q0)**2
    M_SI=M*MSUN_SI
    D_SI=(10**(6))*PC_SI
    inc = angle
    angle = (np.pi/180)*angle
        
    mode2polfac=4*(5/(64*np.pi))**(1/2)
         
    
    hp, hc, tinsp = Ecc_INSP_Eber22(M,q0,e0,l0,fmin,angle,d,delta_t) 
    
        
    #Circular IMR
    sp, sc = get_td_waveform(approximant='SEOBNRv4', mass1=M1, mass2=M2, delta_t=delta_t, f_lower=fmin)

    h22IMR = sp+1j*sc
    
    tshift = -tshift_Hinsp(q0,e0,l0)*M*MTSUN_SI
    
    #print(-tshift_Hinsp(q0,e0,l0))
    tmin = max(tinsp[0]-tshift, sp.sample_times[0])
    
    
    tImr_intrp = np.arange(tmin, sp.sample_times[-1], delta_t)
    tImr = tImr_intrp
    H_intrp = interp1d(sp.sample_times, h22IMR, kind='cubic', fill_value='extrapolate')
    h22Imr = H_intrp(tImr_intrp)
    
    
    
    
    
    #Interpolation Ebersold
    hp_intrp = interp1d(tinsp-tshift, hp, kind='cubic',fill_value='extrapolate')
    hc_intrp = interp1d(tinsp-tshift, hc, kind='cubic',fill_value='extrapolate')
    tEcc_intrp = np.arange(tmin, tinsp[-1]-tshift, delta_t)
    hp_intrp = hp_intrp(tEcc_intrp)
    hc_intrp = hc_intrp(tEcc_intrp)
    tEcc = tEcc_intrp
    hpEcc = hp_intrp
    hcEcc = hc_intrp
    h22Ecc = hpEcc + 1j*hcEcc
    
    phaseEcc = np.unwrap(np.angle(h22Ecc)*2)/2
    phaseImr = np.unwrap(np.angle(h22Imr)*2)/2
    dphase = phaseEcc[0] - phaseImr[0]
    hp_new = real(h22Ecc * exp(-1j * dphase))
    hc_new = imag(h22Ecc * exp(-1j * dphase))
    
    phase_new = np.unwrap(np.angle(hp_new-1j*hc_new)*2)/2
    
    phaseEcc = phase_new 
    phaseImr = phaseImr 
    h22Ecc_new = (hp_new+1j*hc_new)
    
    arg = np.argmin(abs(tEcc-tamp_Hinsp(eta,e0,l0)*M*MTSUN_SI))
    Idxjoin = arg
    
    t_amp = tEcc[Idxjoin] - 500*M*MTSUN_SI
    idxstr = np.argmin(abs(tEcc-t_amp))
    
    #Amplitude Model
    amp=[]
    count=0
    length=Idxjoin-idxstr
    
    for i in range(idxstr,Idxjoin):
        amp.append(((length-count)*abs(h22Ecc_new[i])+count*abs(h22Imr[i]))/length)
        count=count+1
    
    t_model=np.concatenate((tEcc[0:Idxjoin],tImr[Idxjoin:len(tImr)]))
    h22amp=np.concatenate((abs(h22Ecc_new[0:idxstr]),amp))
    h22amp_model=np.concatenate((h22amp,abs(h22Imr[Idxjoin:len(h22Imr)])))    
    
    omegaEcc = (M*MTSUN_SI/delta_t)*(np.gradient(phaseEcc))
    omegaImr = (M*MTSUN_SI/delta_t)*(np.gradient(phaseImr))
    
    tjoin0 = tfreq_Hinsp(eta,e0,l0)
    #print(tjoin0)
    tjoin = tjoin0 * M * MTSUN_SI
    fjoin = np.argmin(abs(tEcc-tjoin))
        
    #frequency model
    tstop = min(tEcc[-1],-30*M*MTSUN_SI)
    lst=np.argmin(abs(tEcc-tstop))
    
    indx=lst - fjoin
    a0 = []
    n = indx-1
    k = 0
    for i in range(fjoin,fjoin+indx):
        a0.append(((n-k)*omegaEcc[i]+k*omegaImr[i])/n)
        k=k+1
    
    f1 = np.concatenate((omegaEcc[0:fjoin],a0))
    frequency_model = np.concatenate((f1,omegaImr[fjoin+indx:len(omegaImr)]))
    phase_f_model = np.cumsum(frequency_model)/(M*MTSUN_SI/delta_t)
    phase_f_model = phase_f_model - phase_f_model[0] + phaseEcc[0]
    
    ll = min(len(h22amp_model),len(phase_f_model))
    hp_f_model = h22amp_model[:ll] * np.cos(phase_f_model[:ll])
    hc_f_model = h22amp_model[:ll] * np.sin(phase_f_model[:ll])
    
    ht = (mode2polfac/4)*(((1+math.cos(angle))**2 * (hp_f_model - 1j*hc_f_model)) + ((1-math.cos(angle))**2 * (hp_f_model + 1j*hc_f_model)))
    hplus = np.real(ht)
    hcross = np.imag(ht)
    
    hp_model_TS = TimeSeries(hplus,delta_t)
    hc_model_TS = TimeSeries(hcross,delta_t)
    #print(hplus)   
    return hp_model_TS, hc_model_TS, t_model#, frequency_model
    